In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os

from tqdm.notebook import tqdm
from scipy.integrate import odeint
from scipy.optimize import fsolve

In [4]:
np.random.seed(42)

NAME = 'assignment3'
DATA_PATH = f'data/{NAME}'
MEDIA_PATH = f'media/{NAME}'
[os.makedirs(path, exist_ok=True) for path in [DATA_PATH, MEDIA_PATH]]
print('Setup complete')

Setup complete


# Question 1: Complex calcium oscillations

### a) & b)

Banger vague question. My favourite.

The proposed system describing intracellular calcium oscillations is characterized by the existence of three calcium stores: the ER, mitochondria and calcium-binding proteins in the cytosol, of which ER is considered to be the main store. We denote their respective calcium concentrations as $[ Ca_{ER}], [ Ca_{m} ], [ Ca_{cyt} ]$. In addition to these three model variables we also consider the concentration of binding sites on the calcium-binding cytosolic proteins. These occur in either unbounded ($Pr$) or bounded ($CaPr$) state. The model neglects import and export through the cell membrane and thus the total concentration of both cellular calcium $Ca_{tot}$ and cytosolic proteins $Pr_{tot}$ is conserved (constant). In summary, the system differentiates four different states of calcium molecules and two states of calcium-binding molecules. This gives rise to the formulas for $Ca_{tot}$ and $Pr_{tot}$ (**ADD REF**).

$$
\begin{equation}
    Ca_{tot} = Ca_{cyt} + \frac{\rho_{ER}}{\beta_{ER}} Ca_{ER} + \frac{\rho_{m}}{\beta_{m}} Ca_{m} + CaPr
\end{equation}
$$

$$
\begin{equation}
    Pr_{tot} = Pr + CaPr
\end{equation}
$$

Parameters $\rho_{ER}, \rho_{m}, \beta_{ER}, \beta_{m}$ represent the compartmental modelling component of the system. Here, $\rho_{ER}$ and $\rho_{m}$ relate to the volume ratio between ER and cytosol and between mitochondria and cytosol, respectively (**ADD REF**). Diffusion of calcium within the ER and mitochondria is assumed to occur rapidly and thus specific spatial modelling of calcium concentration within this components is neglected. Instead, we are interested in relating the total concentration of free calcium in the ER and the mitochondria to the respective total concentrations, which is done through constant factors $\beta_{ER}$ and $\beta_{m}$ (**ADD REF**).

$$
\begin{equation}
    \rho_{ER} \sim \frac{V_{ER}}{V_{cyt}}
\end{equation}
$$

$$
\begin{equation}
    \rho_{m} \sim \frac{V_{m}}{V_{cyt}}
\end{equation}
$$

$$
\begin{equation}
    \beta_{ER} \sim \frac{[Ca_{ER}]}{[Ca_{tot}]}
\end{equation}
$$

$$
\begin{equation}
    \beta_{m} \sim \frac{[Ca_{m}]}{[Ca_{tot}]}
\end{equation}
$$

System's main calcium storage, the endoplasmic reticulum (ER), absorbs calcium in an ATP-dependent process from the surrounding cytosol ($J_{pump}$). This inflow is taken to be a linear function of cytosolic calcium (**ADD REF**) with $k_{pump}$ rate constant related to the presence of ATP-ases, enzymes catalyzing the decomposition of ATP into ADP. Efflux of calcium from the ER occurs in two ways. A calcium-induced calcium release (CICR) mechanism moderates the efflux of calcium ions $Ca^{2+}$ through excitable membrane channels ($J_{ch}$) (**ADD REF**). This process is dependent on the availability of cytosolic calcium, which is modelled by a Hill equation with coefficient $2$ and half-saturation constant $K_1$ as well as maximal permeability of $Ca^{2+}$ membrane channels, described by constant $k_{ch}$. Additionally to that we also distinguish a separate calcium ion efflux into the surrounding cytosol, the leak flux $J_{leak}$ (**ADD REF**) with rate constant $k_{leak}$. Both of these effluxes $J_{ch}$ and $J_{leak}$ depend on ER transmembrane potential, but this dependency is neglected for simplification purposes. Time-dependent change of free calcium concentration in the ER is thus governed by influx $J_{pump}$ and effluxes $J_{ch}$ and $J_{leak}$ and all of these fluxes are also dependent on volume and concetration related factors $\beta_{ER}, \rho_{ER}$ (**ADD REF**).

$$
\begin{equation}
    J_{pump} = k_{pump} Ca_{cyt}
\end{equation}
$$

$$
\begin{equation}
    J_{ch} = k_{ch} \frac{Ca_{cyt}^2}{K_1^2 + Ca_{cyt}^2} (Ca_{ER} - Ca_{cyt})
\end{equation}
$$

$$
\begin{equation}
    J_{leak} = k_{leak} (Ca_{ER} - Ca_{cyt})
\end{equation}
$$

$$
\begin{equation}
    \frac{d Ca_{ER}}{dt} = \frac{\beta_{ER}}{\rho_{ER}} (J_{pump} - J_{ch} - J_{leak})
\end{equation}
$$

The role of mitochondria in the system is the one of a slowly-leaking storage of calcium absorbed from the ER. This slow (up to $100$ times slower than the uptake) leakage then serves as a supply for intermediate exchange between the ER and the cytosolic proteins, which causes bursting calcium oscillations. A majority ($\approx 80 \%$) of calcium released from the ER is initially rapidly sequestered into the mitochondria by mitochondrial uniporters ($J_{in}$) (**ADD REF**). This inflow is modelled with a step-like kinetics and the dependence on cytosolic calcium is modelled through a Hill equation with coefficient $8$, and $K_2$ half-saturation constant. Additional factor $k_{in}$ is introduced to account for maximal permeability of uniporters. The slow leakeage $J_{out}$ (**ADD REF**) occurs in a very low-conductance state through permeability transition pores (PTPs) located in the inner membrane. The constant $k_{out}$ is the maximal rate for calcium flux through $Na^{+} / Ca^{2+}$ exchangers and PTPs in a low-conductance state and the rate constant $k_m$ accounts for non-specific leak flux. The time-dependent change of free calcium concentration in the mitochondria is thus governed by influx $J_{in}$ and efflux $J_{out}$ (**ADD REF**). Volume and concentration related factors again influence the final outcome.

$$
\begin{equation}
    J_{in} = k_{in} \frac{Ca_{cyt}^8}{K_2^8 + Ca_{cyt}^8}
\end{equation}
$$

$$
\begin{equation}
    J_{out} = (k_{out} \frac{Ca_{cyt}^2}{K_3^2 + Ca_{cyt}^2} + k_m) Ca_{m}
\end{equation}
$$

$$
\begin{equation}
    \frac{d Ca_{m}}{dt} = \frac{\beta_{m}}{\rho_{m}} (J_{in} - J_{out})
\end{equation}
$$

All of the five introduced fluxes $J_{pump}, J_{ch}, J_{leak}, J_{in}, J_{out}$ are mediated through the cytosol surrounding the ER and mitochondria. The fluxes are complementary in the sense that calcium inflowing into the ER or mitochondrium is absorbed from the cytosol and, vice versa, calcium ejected from the ER or mitochondria will freely flow in surrounding cytosol. In addition to these calcium fluxes, a reaction takes place between free calcium molecules and proteins present in the cytosol. Free calcium molecules $Ca$ bind to the sites on free cytosolic proteins $Pr$ resulting in a $CaPr$ complex. As the binding is reversible, a backward reaction also takes place. Related to this binding and unbinding process are forward and backward reaction constants $k_{+}$ and $k_{-}$. The total time-dependent change of free calcium concentration in the cytosol is thus governed by five calcium fluxes and a reversible reaction with cytosolic proteins (**ADD REF**). Since the compartment modelling constants $\beta$ and $\rho$ are expressed as ratios with regards to cytosol, these terms are not present in the final formula.

$$
\begin{equation}
    \frac{d Ca_{cyt}}{dt} = J_{ch} + J_{leak} - J_{pump} + J_{out} - J_{in} + k_{-} [CaPr] - k_{+} [Ca_{cyt}] [Pr] \\
\end{equation}
$$



### c)

The $Ca^{2+}$ channels govern the channel efflux $J_{ch}$ of calcium ions out of the ER. A genetic mutation would shut down these channels resulting in no pearmeability of $Ca^{2+}$ membrane channels $k_{ch}$ and thus no efflux $J_{ch}$:

$$ k_{ch} = 0 \implies J_{ch} = 0 . $$

As a direct consequence, the model equations would become:

$$
\begin{align*}
    \frac{d Ca_{cyt}}{dt} &= J_{leak} - J_{pump} + J_{out} - J_{in} + k_{-} [CaPr] - k_{+} [Ca_{cyt}] [Pr] \\
    \frac{d Ca_{m}}{dt} &= \frac{\beta_{m}}{\rho_{m}} (J_{in} - J_{out}) \\
    \frac{d Ca_{ER}}{dt} &= \frac{\beta_{ER}}{\rho_{ER}} (J_{pump} - J_{leak})
\end{align*}
$$

Since main calcium efflux from the ER is mediated through these membrane channels, we will observe an accumuluation of calcium in the ER. Without channel efflux, the calcium can never be rapidly sequestered in mitochondria, which in turn is resposnible for bursting calcium oscillations through the slow-leakage efflux $J_{out}$. Thus, the mutations might cause a complete lack of complex oscillations in the system's dynamics behaviour.

**TODO:** FINISH THIS (?)

### d)

From the conservation of total cellular calcium:

$$ CaPr = Ca_{tot} - Ca_{cyt} - \frac{\rho_{ER}}{\beta_{ER}} Ca_{ER} - \frac{\rho_{m}}{\beta_{m}} Ca_{m} $$

From the conservation of bound and unbound proteins:

$$ Pr = Pr_{tot} - CaPr = Pr_{tot} - Ca_{tot} + Ca_{cyt} + \frac{\rho_{ER}}{\beta_{ER}} Ca_{ER} + \frac{\rho_{m}}{\beta_{m}} Ca_{m} $$

Plugging this into the ODE system we get:

$$
\begin{align*}
    \frac{d Ca_{cyt}}{dt} &= J_{ch} J_{leak} - J_{pump} + J_{out} - J_{in} + \\ 
    & + k_{-} (Ca_{tot} - Ca_{cyt} - \frac{\rho_{ER}}{\beta_{ER}} Ca_{ER} - \frac{\rho_{m}}{\beta_{m}} Ca_{m}) - \\
    & - k_{+} [Ca_{cyt}] (Pr_{tot} - Ca_{tot} + Ca_{cyt} + \frac{\rho_{ER}}{\beta_{ER}} Ca_{ER} + \frac{\rho_{m}}{\beta_{m}} Ca_{m}) \\
    \frac{d Ca_{m}}{dt} &= \frac{\beta_{m}}{\rho_{m}} (J_{in} - J_{out}) \\
    \frac{d Ca_{ER}}{dt} &= \frac{\beta_{ER}}{\rho_{ER}} (J_{pump} - J_{ch} - J_{leak})
\end{align*}
$$

None of the flux terms depends on any of the protein concentrations, thus the system was reduced to three independet variables $Ca_{cyt}, Ca_{ER}$ and $Ca_{m}$.